In [1]:
import os
# https://discuss.pytorch.org/t/how-to-change-the-default-device-of-gpu-device-ids-0/1041/24
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]=f"{1}"

import torch
print(f"current_device:{torch.cuda.current_device()}")
print(f"device_count:{torch.cuda.device_count()}")
device = torch.device("cuda")


import sys
import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from ddpg import DDPGagent
from ppo.PPO_continuous import PPO,Memory
# from ddpg.utils import NormalizedEnv
from flexipod_env import FlexipodEnv

env = FlexipodEnv(dof = 12)
# env = NormalizedEnv(env)

############## Hyperparameters ##############
# env_name = "BipedalWalker-v3"
env_name = "flexipod"
render = True
solved_reward = 1500        # stop training if avg_reward > solved_reward
log_interval = 80           # print avg reward in the interval
# log_interval = 2           # print avg reward in the interval

max_episodes = 20000        # max training episodes
max_timesteps = 1500        # max timesteps in one episode

# update_timestep = 4000      # update policy every n timesteps
update_timestep = 3000      # update policy every n timesteps


action_std = 0.5            # constant std for action distribution (Multivariate Normal)
# action_std = 1.0          # constant std for action distribution (Multivariate Normal)
K_epochs = 80               # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr = 0.0002                 # parameters for Adam optimizer
betas = (0.9, 0.999)

random_seed = None
#############################################
# creating environment
# env = gym.make(env_name)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

current_device:0
device_count:1


In [2]:
from torch.utils.tensorboard import SummaryWriter
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/soft12dof_experiment_8')

In [3]:
# env = FlexipodEnv(dof = 12)
# self = env
# msg_rec,_,_,_ = env.step()

In [3]:
if random_seed:
    print("Random Seed: {}".format(random_seed))
    torch.manual_seed(random_seed)
    env.seed(random_seed)
    np.random.seed(random_seed)

memory = Memory()
ppo = PPO(state_dim, action_dim, action_std, lr, betas, gamma, K_epochs, eps_clip)
print(lr,betas)

max_avg_length = 0
# checkpoint = ppo.load(f'./PPO_continuous_{env_name}_best.pth')
# max_avg_length = checkpoint["avg_length"]

# logging variables
running_reward = 0
avg_length = 0
# max_avg_length = 0
time_step = 0

# training loop
for i_episode in range(1, max_episodes+1):
    state = env.reset()
    for t in range(max_timesteps):
        time_step +=1
        # Running policy_old:
        action = ppo.select_action(state, memory)
        state, reward, done, _ = env.step(action)

        # Saving reward and is_terminals:
        memory.rewards.append(reward)
        memory.is_terminals.append(done)

        # update if its time
        if time_step % update_timestep == 0:
            ppo.update(memory)
            memory.clear_memory()
            time_step = 0
        running_reward += reward
        if render:
            env.render()
        if done:
            break

    avg_length += t

    # save every 500 episodes
    if i_episode % 500 == 0:
        ppo.save(f'./PPO_continuous_{env_name}.pth',avg_length=avg_length)

    # logging
    if i_episode % log_interval == 0:
        avg_length = avg_length/log_interval
        running_reward = running_reward/log_interval
        writer.add_scalar("avg_length/train", avg_length, i_episode)
        writer.add_scalar("running_reward/train", running_reward, i_episode)
        
        # stop training if avg_reward > solved_reward
        if running_reward > (log_interval*solved_reward):
            print("########## Solved! ##########")
            ppo.save(f'./PPO_continuous_solved_{env_name}.pth',avg_length=avg_length)
            break
            
        if avg_length>max_avg_length:
            max_avg_length = avg_length
            ppo.save(f'./PPO_continuous_{env_name}_best.pth',avg_length=avg_length)
        elif np.random.random()<0.1:# 50% chance 
            checkpoint = ppo.load(f'./PPO_continuous_{env_name}_best.pth')
            print(f"load old best,avg_length={checkpoint['avg_length']}")# restart

        print(f'Episode {i_episode} \t Avg length: {avg_length:.0f} \t Avg reward: {running_reward:.0f}')
        running_reward = 0
        avg_length = 0
        
env.pause()

0.0002 (0.9, 0.999)
Episode 80 	 Avg length: 110 	 Avg reward: 138
Episode 160 	 Avg length: 139 	 Avg reward: 174
Episode 240 	 Avg length: 157 	 Avg reward: 198
Episode 320 	 Avg length: 167 	 Avg reward: 211
Episode 400 	 Avg length: 187 	 Avg reward: 237
Episode 480 	 Avg length: 207 	 Avg reward: 263
Episode 560 	 Avg length: 250 	 Avg reward: 320
Episode 640 	 Avg length: 272 	 Avg reward: 348
Episode 720 	 Avg length: 319 	 Avg reward: 408
Episode 800 	 Avg length: 305 	 Avg reward: 391
Episode 880 	 Avg length: 313 	 Avg reward: 402
Episode 960 	 Avg length: 339 	 Avg reward: 435
Episode 1040 	 Avg length: 370 	 Avg reward: 474
Episode 1120 	 Avg length: 353 	 Avg reward: 453
Episode 1200 	 Avg length: 332 	 Avg reward: 426
Episode 1280 	 Avg length: 321 	 Avg reward: 412
Episode 1360 	 Avg length: 388 	 Avg reward: 499
Episode 1440 	 Avg length: 415 	 Avg reward: 534
Episode 1520 	 Avg length: 362 	 Avg reward: 465
Episode 1600 	 Avg length: 421 	 Avg reward: 543
Episode 1680 

KeyboardInterrupt: 

In [4]:
# env.reset()
self = env

In [5]:
self.reset()

KeyError: 'com_pos'

In [7]:
msg_rec = self.receive()

In [13]:
msg_rec[0]

[14,
 526.0750122070312,
 [1.5707980394363403,
  -1.5707910060882568,
  1.1583278179168701,
  -1.1581804752349854,
  1.5618208646774292,
  0.00015313680341932923,
  -1.5617340803146362,
  0.00014471155009232461,
  0.00017664824554231018,
  -0.4020088016986847,
  -0.00022034731227904558,
  -0.4015836715698242],
 [1.2466489351936616e-07,
  -6.653957029811863e-08,
  -3.5676137486007065e-05,
  3.467056012596004e-05,
  7.091095994837815e-07,
  1.28126725940092e-06,
  -7.667101726838155e-07,
  1.2046399433529587e-06,
  -3.122644329778268e-07,
  5.897355731576681e-05,
  -1.964820768307618e-07,
  6.117679731687531e-05],
 [-2.5745034690771718e-06,
  -8.177499694284052e-06,
  0.006205763202160597,
  -0.006430943496525288,
  0.013713488355278969,
  -0.0002339880302315578,
  -0.013846149668097496,
  -0.00022111446014605463,
  -0.0002698956523090601,
  -0.009776351042091846,
  0.0003366677847225219,
  -0.01042591966688633],
 [0.1014338880777359,
  -7.22313197911717e-05,
  0.9948422908782959,
  -0.0

In [13]:
# writer.add_scalar("baseline_length/train", 200, 0)
# writer.add_scalar("baseline_length/train", 200, log_interval)

In [5]:
memory_2 = Memory()

# action_std=.5
ppo = PPO(state_dim, action_dim, action_std, lr, betas, gamma, K_epochs, eps_clip)
ppo.load(f'./PPO_continuous_{env_name}_best.pth')

time_steps = []
durations = []
for k in range(20):
    state = env.reset()
    for t in range(max_timesteps):
        # Running policy_old:
        action = ppo.select_action(state, memory_2)
#         action = ppo.select_action(state, memory)
        state, reward, done, info = env.step(action)
#         state, reward, done, _ = env.step()

        if done:
            time_steps.append(t)
            episode_duration= info['episode_duration']
            durations.append(episode_duration)
            print(f"time steps:{t} ,duration: {episode_duration:.2f}[s], fps:{t/episode_duration:.0f}")
            break
            
print(f"mean time steps:{np.mean(time_steps)} ,duration: {np.mean(durations)}[s]")

time steps:364 ,duration: 4.58[s], fps:80
time steps:437 ,duration: 5.50[s], fps:79
time steps:633 ,duration: 7.84[s], fps:81
time steps:891 ,duration: 11.02[s], fps:81
time steps:654 ,duration: 8.05[s], fps:81
time steps:1111 ,duration: 13.87[s], fps:80
time steps:1279 ,duration: 15.86[s], fps:81
time steps:295 ,duration: 3.81[s], fps:77
time steps:252 ,duration: 3.15[s], fps:80
time steps:979 ,duration: 12.14[s], fps:81
time steps:1423 ,duration: 17.66[s], fps:81
time steps:487 ,duration: 6.20[s], fps:79
time steps:416 ,duration: 5.21[s], fps:80
time steps:310 ,duration: 3.84[s], fps:81
time steps:687 ,duration: 8.54[s], fps:80
time steps:439 ,duration: 5.46[s], fps:80
time steps:629 ,duration: 7.86[s], fps:80
time steps:282 ,duration: 3.49[s], fps:81
time steps:197 ,duration: 2.47[s], fps:80
mean time steps:619.2105263157895 ,duration: 7.713610197368421[s]
